# Arrivals in "city" for tomorrow :

#### Importing Libraries

In [1]:
path = "C:/Users/Kosta/OneDrive/Desktop/Data_Science/5_APIS"
import sys
sys.path.insert(1, path)

In [2]:
from api_keys import *

In [3]:
from IPython.display import JSON
import json
import pandas as pd
import requests
import datetime

In [4]:
def get_cities_dataframe():
    cities = pd.DataFrame({'City': ['Istanbul','Moscow','Berlin'], 'Country': ['Turkey','Russian Federation (the)','Germany'], 
    'Country_code': ['TR','RU','DE'],'CityCountry' : ["Istanbul, TR","Moscow, RU","Berlin, DE"],
    'Country_short' : ['TR','RU','DE'], 'icao' : ["LTFM","UUDD","EDDB"]})
    return cities[:3]
    
#     cities = pd.DataFrame({'City': ['Istanbul','Moscow','Berlin'], 'Country': ['Turkey','Russian Federation (the)','Germany'],'Population' : [15462452,12195221,3748148], 
#     'Country_code': ['TR','RU','DE'],'CityCountry' : ["Istanbul, TR","Moscow, RU","Berlin, DE"], 'name' : ["Istanbul Airport","Domodedovo International Airport","Berlin Brandenburg Airport"],
#     'Countryshort' : ['TR','RU','DE'], 'icao' : ["LTFM","UUDD","EDDB"]})
    
# def get_cities_dataframe():
#     cities = pd.read_csv(path_two + '\cities.csv')
#     return cities[:3]

In [5]:
# cities = pd.DataFrame({'City': ['Istanbul','Moscow','Berlin'], 'Country': ['Turkey','Russian Federation (the)','Germany'],'Population' : [15462452,12195221,3748148], 
#      'Country_code': ['TR','RU','DE'],'CityCountry' : ["Istanbul, TR","Moscow, RU","Berlin, DE"], 'name' : ["Istanbul Airport","Domodedovo International Airport","Berlin Brandenburg Airport"],
#     'Countryshort' : ['TR','RU','DE'], 'icao' : ["LTFM","UUDD","EDDB"]})
# cities
cities = get_cities_dataframe()
cities

,City,Country,Country_code,CityCountry,Country_short,icao
0,Istanbul,Turkey,TR,"Istanbul, TR",TR,LTFM
1,Moscow,Russian Federation (the),RU,"Moscow, RU",RU,UUDD
2,Berlin,Germany,DE,"Berlin, DE",DE,EDDB


## Flight schedule :

In [6]:
# These are only the very first hours from 07:00 till 19:00
# Between 00:00 and 07:00 there were no flights found in Berlin.

def get_jsons_from_AerodataboxAPI(icao):
    today = datetime.date.today()
    tomorrow = str(today + datetime.timedelta(days=1))    
    times = [["00:00","11:59"],["12:00","23:59"]]
    jsons = []
    for time in times:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
        querystring = {"direction":"Arrival","withPrivate":"true"}
        headers = {
            'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
            'x-rapidapi-key': flight_API_key
            }
        jsons.append(requests.request("GET", url, headers=headers, params=querystring).json())
    return jsons

### It works ! 

In [7]:
# NO need to repeat this
# response_one = get_jsons_from_AerodataboxAPI('EDDB')
JSON(response_one)
# response_two = (get_jsons_from_AerodataboxAPI('EDDB')).json()
# response_two

<IPython.core.display.JSON object>

In [9]:
def create_dictionary_from_json(jsons):
    dictionary = {"From_location":[],"Arrival_at":[], "Terminal":[],"Airline":[],"Flight_number":[],"Airport_code":[]}
    for resp in jsons:
        for flight in resp['arrivals']:
            try:
                dictionary["From_location"].append(flight['movement']["airport"]['name'])
            except:
                dictionary["From_location"].append("Unknown")
            try:
                dictionary["Arrival_at"].append(flight['movement']['scheduledTimeLocal'][:-6])
            except:
                dictionary["Arrival_at"].append("Unknown")
            try:
                dictionary["Terminal"].append(flight['movement']['terminal'])
            except:
                dictionary["Terminal"].append("Unknown")
            try:
                dictionary["Airline"].append(flight['airline']['name'])
            except:
                dictionary["Airline"].append("Unknown")
            try:
                dictionary["Flight_number"].append(flight['number'])
            except:
                dictionary["Flight_number"].append("Unknown")
            try:
                dictionary["Airport_code"].append(flight['movement']["airport"]["icao"])
            except:
                dictionary["Airport_code"].append("Unknown")
    return dictionary 

In [10]:
def create_dataframe_from_dictionaries(dictionary):
    flights = pd.DataFrame.from_dict(dictionary)
    return flights

In [11]:
def get_all_info(cities):
    big_table = pd.DataFrame()
    for index, city in cities.iterrows():
        jsons = get_jsons_from_AerodataboxAPI(city['icao'])
        dictionary = create_dictionary_from_json(jsons)
        dataframe = create_dataframe_from_dictionaries(dictionary)
        dataframe = dataframe.assign(CityCountry = city['CityCountry'])
        dataframe = dataframe.assign(icao = city['icao'])
        dataframe["Arrival_at"] = pd.to_datetime(dataframe["Arrival_at"])
#         dataframe["Arrival_UTC_time"] = pd.to_datetime(dataframe["Arrival_UTC_time"])
    big_table = big_table.append(dataframe,ignore_index=True)
    return big_table

In [12]:
def create_mysql_table(big_table):
    schema="gans"
    host="127.0.0.1"
    user="root"
    password=sql_pass
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    big_table.to_sql('flights',con=con,if_exists='append',index=False)

In [45]:
create_mysql_table(get_all_info(get_cities_dataframe()))

In [13]:
all_info= get_all_info(get_cities_dataframe())

In [14]:
all_info

,From_location,Arrival_at,Terminal,Airline,Flight_number,Airport_code,CityCountry,icao
0,Frankfurt-am-Main,2022-01-23 07:55:00,1,Lufthansa,LH 170,EDDF,"Berlin, DE",EDDB
1,Frankfurt-am-Main,2022-01-23 08:55:00,1,Lufthansa,LH 172,EDDF,"Berlin, DE",EDDB
2,Frankfurt-am-Main,2022-01-23 09:55:00,1,Lufthansa,LH 176,EDDF,"Berlin, DE",EDDB
3,Frankfurt-am-Main,2022-01-23 11:55:00,1,Lufthansa,LH 180,EDDF,"Berlin, DE",EDDB
4,Munich,2022-01-23 08:10:00,1,Lufthansa,LH 1928,EDDM,"Berlin, DE",EDDB
...,...,...,...,...,...,...,...,...
149,Kiev,2022-01-23 20:30:00,0,Wizz Air,W6 6113,UKKK,"Berlin, DE",EDDB
150,Kaliningrad,2022-01-23 13:10:00,1,Pegas Fly,EO 2267,UMKK,"Berlin, DE",EDDB
151,Moscow,2022-01-23 12:55:00,1,Aeroflot,SU 2568,UUEE,"Berlin, DE",EDDB
152,Moscow,2022-01-23 17:55:00,1,Aeroflot,SU 2664,UUEE,"Berlin, DE",EDDB


### JSON display 1

In [16]:
response_one

[{'arrivals': [{'movement': {'airport': {'icao': 'EDDF',
      'iata': 'FRA',
      'name': 'Frankfurt-am-Main'},
     'scheduledTimeLocal': '2022-01-23 11:55+01:00',
     'scheduledTimeUtc': '2022-01-23 10:55Z',
     'terminal': '1',
     'quality': ['Basic']},
    'number': 'LH 180',
    'status': 'Unknown',
    'codeshareStatus': 'IsCodeshared',
    'isCargo': False,
    'aircraft': {'model': 'Airbus A320'},
    'airline': {'name': 'Lufthansa'}},
   {'movement': {'airport': {'icao': 'EDDM', 'iata': 'MUC', 'name': 'Munich'},
     'scheduledTimeLocal': '2022-01-23 08:10+01:00',
     'scheduledTimeUtc': '2022-01-23 07:10Z',
     'terminal': '1',
     'quality': ['Basic']},
    'number': 'LH 1928',
    'status': 'Unknown',
    'codeshareStatus': 'IsCodeshared',
    'isCargo': False,
    'aircraft': {'model': 'Airbus A320'},
    'airline': {'name': 'Lufthansa'}},
   {'movement': {'airport': {'icao': 'EDDF',
      'iata': 'FRA',
      'name': 'Frankfurt-am-Main'},
     'scheduledTimeLocal

In [18]:
# # flights between 07:00 - 19:00
JSON(response_one)

<IPython.core.display.JSON object>

### JSON display 2

In [41]:
# # flights between 19:01-23:59
# JSON(response_two) # Run this only if you run the 2nd response from get request

<IPython.core.display.JSON object>

## Dictionary

### Creating the data Frame of the flights we are interested in

In [102]:
# flights = pd.DataFrame.from_dict(dictionary)
# flights

,From,Arrival at :,Terminal,Airline,Flight number,Airport code
0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
1,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
2,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
3,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
4,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
...,...,...,...,...,...,...
715,Milan,2021-11-25 22:30,1,easyJet,U2 4678,LIMC
716,Bâle/Mulhouse,2021-11-25 22:20,1,easyJet,U2 5640,LFSB
717,Copenhagen,2021-11-25 22:30,1,easyJet,U2 5690,EKCH
718,Palma De Mallorca,2021-11-25 22:30,1,easyJet,U2 5808,LEPA


### Including only flights arriving from "Barcelona"

In [94]:
# arr_from_Barc =flights.query('From == "Barcelona"')
# arr_from_Barc.reset_index().head(10)

,index,From,Arrival at :,Terminal,Airline,Flight number,Airport code
0,252,Barcelona,2021-11-25 13:55+01:00,1,easyJet,U2 5122,Unknown
1,259,Barcelona,2021-11-25 14:10+01:00,1,Vueling,VY 1882,LEBL
2,283,Barcelona,2021-11-25 17:00+01:00,Unknown,Ryanair,FR 133,LEBL
3,432,Barcelona,2021-11-25 13:55,1,easyJet,U2 5122,Unknown
4,439,Barcelona,2021-11-25 14:10,1,Vueling,VY 1882,LEBL
5,463,Barcelona,2021-11-25 17:00,Unknown,Ryanair,FR 133,LEBL


### Including only flights without "Unknown"

In [101]:
# flights_known =flights.query('From != "Unknown"')
# flights_known.reset_index()

,index,From,Arrival at :,Terminal,Airline,Flight number,Airport code
0,180,Riga,2021-11-25 07:55+01:00,1,Air Baltic,BT 211,EVRA
1,181,Cologne,2021-11-25 07:20+01:00,1,Eurowings,EW 12,EDDK
2,182,Stuttgart,2021-11-25 07:30+01:00,1,Eurowings,EW 2002,EDDS
3,183,Duesseldorf,2021-11-25 07:35+01:00,1,Eurowings,EW 9048,EDDL
4,184,Frankfurt-am-Main,2021-11-25 07:55+01:00,1,Lufthansa,LH 170,EDDF
...,...,...,...,...,...,...,...
535,715,Milan,2021-11-25 22:30,1,easyJet,U2 4678,LIMC
536,716,Bâle/Mulhouse,2021-11-25 22:20,1,easyJet,U2 5640,LFSB
537,717,Copenhagen,2021-11-25 22:30,1,easyJet,U2 5690,EKCH
538,718,Palma De Mallorca,2021-11-25 22:30,1,easyJet,U2 5808,LEPA
